# 1. Exploration.

In [1]:
import numpy as np
import pandas as pd
import mytools as mt
import spacy
import re

### 1.1 EXTRACTION ET CHARGEMENT DES QUESTIONS DE STACKOVERFLOW

Nous allons extraire 50000 questions qui ont les caractéristiques suivantes :

- plus de 25000 vues ;
- au moins une réponse ;
- au moins 5 Tags ;
- classées par score décroissant.

Voici la requête SQL effectuée sur le site https://data.stackexchange.com/stackoverflow/query/edit/1838136 (Les données ont été enregistrées dans un fichier csv) :

Chargeons les questions dans un DataFrame Pandas :

In [4]:
data = pd.read_csv('data_stackoverflow.csv')
data.head()

,Id,Title,Body,Tags,Score,ViewCount,AnswerCount
0,11227809,Why is processing a sorted array faster than p...,"<p>In this C++ code, sorting the data (<em>bef...",<java><c++><performance><cpu-architecture><bra...,27243,1880186,25
1,2003505,How do I delete a Git branch locally and remot...,<p>Failed Attempts to Delete a Remote Branch:<...,<git><version-control><git-branch><git-push><g...,20386,11483075,41
2,1642028,What is the '-->' operator in C/C++?,"<p>After reading <a href=""http://groups.google...",<c++><c><operators><code-formatting><standards...,10156,1010497,26
3,1125968,"How do I force ""git pull"" to overwrite local f...",<p>How do I force an overwrite of local files ...,<git><version-control><overwrite><git-pull><gi...,9647,8511461,53
4,79923,What and where are the stack and heap?,<ul>\n<li>What are the stack and heap?</li>\n<...,<data-structures><memory-management><heap-memo...,9420,1931849,31


In [5]:
data.shape

(50000, 7)

In [6]:
data.set_index('Id', inplace=True)
data.head()

,Title,Body,Tags,Score,ViewCount,AnswerCount
Id,,,,,,
11227809,Why is processing a sorted array faster than p...,"<p>In this C++ code, sorting the data (<em>bef...",<java><c++><performance><cpu-architecture><bra...,27243,1880186,25
2003505,How do I delete a Git branch locally and remot...,<p>Failed Attempts to Delete a Remote Branch:<...,<git><version-control><git-branch><git-push><g...,20386,11483075,41
1642028,What is the '-->' operator in C/C++?,"<p>After reading <a href=""http://groups.google...",<c++><c><operators><code-formatting><standards...,10156,1010497,26
1125968,"How do I force ""git pull"" to overwrite local f...",<p>How do I force an overwrite of local files ...,<git><version-control><overwrite><git-pull><gi...,9647,8511461,53
79923,What and where are the stack and heap?,<ul>\n<li>What are the stack and heap?</li>\n<...,<data-structures><memory-management><heap-memo...,9420,1931849,31


In [7]:
data.shape

(50000, 6)

In [9]:
mt.stats(data)

,count,mean,std,min,25%,50%,75%,max
Score,50000.0,75.218,283.148,5.0,15.0,28.0,61.0,27243.0
ViewCount,50000.0,100313.430,218662.016,25001.0,33291.0,48795.0,89935.5,11483075.0
AnswerCount,50000.0,6.047,5.788,1.0,3.0,4.0,7.0,141.0


In [15]:
mt.valeurs_manquantes(data)

,effectif,taux


Il n'y a pas de valeurs aberrantes ni de valeurs manquantes.

### 1.2 PRÉTRAITEMENT DES FEATURES TEXTUELLES

Nous allons utiliser les librairies `spaCy` et `re` pour créer une fonction de prétraitement de texte.

La fonction **process_text** ci-dessous permet d'extraire les mots d'un texte en supprimant les caractères non-alphabétiques (sauf les +, - et _) à l'aide d'une expression régulière et les mots trop courants (stopwords). Elle possède trois arguments :
- ***text*** : La chaine de caractères que l'on souhaite traiter ;
- ***allowed_words*** (optionnel) : Liste de mots autorisés. Les mots qui ne sont pas dans cette liste sont exclus. Par défaut, allowed_words = None ;
- ***unique*** (optionnel) : *True* ou *False*. Supprime les doublons. Par défaut, unique=False.

Commençons par charger le modèle linguistique large anglais (Les questions sont écrites en anglais) de spaCy :

In [19]:
# Charger un modèle linguistique anglais
nlp = spacy.load("en_core_web_lg")

In [26]:
def process_text(text, allowed_words=None, unique=False):

    # Suppression des caractères qui ne sont pas des lettres ou un des symboles (+, -, _) :
    text = re.sub(r'[^a-zA-Z\s\+\-_]', ' ', text)

    # Application du modèle linguistique au texte :
    doc = nlp(text)
    
    # Lemmatisation des mots, exclusion des stopwords, des mots de longueur inférieure à 3 et éventuellement, des mots non-autorisés :
    if allowed_words:
        words = [token.lemma_.lower() for token in doc\
                 if not token.is_stop and len(token.text) >= 3 and token.lemma_.lower() in allowed_words]
    else:
        words = [token.lemma_.lower() for token in doc if not token.is_stop and len(token.text) >= 3]

    # Suppression des doublons.
    if unique == True:
        words = list(set(words))
    
    return words

In [28]:
from time import time

In [30]:
debut = time()

data.Tags = data.Tags.apply(lambda x: process_text(x, unique=True))

fin = time()

duree = fin - debut
print(f"Durée d'exécution : {round(duree/60)} min")

Durée d'exécution : 4 min


In [31]:
debut = time()

data.Title = data.Title.apply(process_text)
data.Body = data.Body.apply(process_text)

fin = time()

duree = fin - debut
print(f"Durée d'exécution : {round(duree/60)} min")

Durée d'exécution : 33 min


In [34]:
data.head()

,Title,Body,Tags,Score,ViewCount,AnswerCount
Id,,,,,,
11227809,"[process, sort, array, fast, process, unsorted...","[c++, code, sort, datum, time, region, make, p...","[cpu, branch, java, architecture, performance,...",27243,1880186,25
2003505,"[delete, git, branch, locally, remotely]","[fail, attempt, delete, remote, branch, pre, c...","[branch, git, version, push, control, remote]",20386,11483075,41
1642028,"[operator, c++]","[read, href, http, group, google, com, group, ...","[operator, compliance, code, standard, c++, fo...",10156,1010497,26
1125968,"[force, git, pull, overwrite, local, file]","[force, overwrite, local, file, code, git, pul...","[pull, fetch, git, version, overwrite, control]",9647,8511461,53
79923,"[stack, heap]","[stack, heap, locate, physically, computer, me...","[memory, dynamic, allocation, structure, manag...",9420,1931849,31


Convertissons toutes les listes en chaines de caractères afin de les rendre mieux exploitables par les algorithmes ultérieurs :

In [37]:
# Convertir les listes de chaînes de caractères en textes :
data.Title = data.Title.apply(lambda x: ' '.join(x))
data.Body = data.Body.apply(lambda x: ' '.join(x))
data.Tags = data.Tags.apply(lambda x: ' '.join(x))

In [39]:
data.head()

,Title,Body,Tags,Score,ViewCount,AnswerCount
Id,,,,,,
11227809,process sort array fast process unsorted array,c++ code sort datum time region make primary l...,cpu branch java architecture performance c++ p...,27243,1880186,25
2003505,delete git branch locally remotely,fail attempt delete remote branch pre class la...,branch git version push control remote,20386,11483075,41
1642028,operator c++,read href http group google com group comp lan...,operator compliance code standard c++ format,10156,1010497,26
1125968,force git pull overwrite local file,force overwrite local file code git pull code ...,pull fetch git version overwrite control,9647,8511461,53
79923,stack heap,stack heap locate physically computer memory e...,memory dynamic allocation structure management...,9420,1931849,31


Enfin, nous allons créer de façon aléatoire un dataset d'entraînement et un dataset de validation avec 50% des données chacun :

In [42]:
from sklearn.model_selection import train_test_split

In [44]:
data_train, data_val = train_test_split(data, test_size=0.5, random_state=88)

On exporte ces deux datasets dans des fichiers csv :

In [47]:
data_train.to_csv('questions_pretraitees.csv')

In [49]:
data_val.to_csv('questions_pretraitees_validation.csv')